# Modelo 4 - Heurística Final

### Leer los datos

In [1]:
import pandas as pd

#Funcion para leer los datos de los pacientes
def leer_datos():
    # Leer el archivo de excel
    df = pd.read_excel('DT4.xlsx', sheet_name='Pacientes') #!!!! Se cambia el nombre de la hoja
    df1 = pd.read_excel("DT4.xlsx", sheet_name="Medicos") #!!!! Se cambia el nombre de la hoja
    df3 = pd.read_excel("DT4.xlsx", sheet_name="Clientes")
    
    # Crear un diccionario para almacenar los datos de los pacientes y otro para los medicos
    pacientesDict = {}
    medicos = {}
    Clientes = {}
    
    # diccionario con los tipos de cliente y el tiempo de atencion esperado
    for index, row in df3.iterrows():
        llave = row["Tipo"]
        valor = row["Tiempo"]
        Clientes[llave] = valor


    # Iterar sobre las filas del DataFrame
    for index, row in df.iterrows():
        llave = row["ID Servicio"]
        valor1 = row["Llegada"]
        valor2 = row["Atencion"]
        valor3 = row["Latitud"]
        valor4 = row["Longitud"]
        valor5 = row["Entidad"]

        # Agregar valores a la lista correspondiente en el diccionario
        if llave in pacientesDict:
            pacientesDict[llave].append([valor1, valor2, valor3, valor4, valor5])
        else:
            pacientesDict[llave] = [int(valor1), int(valor2), valor3, valor4, str(valor5)]
    pacientes = pacientesDict

    # Iterar sobre las filas del DataFrame
    for index, row in df1.iterrows():
        llave = row["ID Medico"]
        Disponibiidad = 0 
        UltOcupado = 0
        NumSol = 0
        lat = 4.681230925773865
        lon = -74.06200456196707
        Inicioj = row["Inicio Jornada"]
        Finj = row["Fin Jornada"]
    
        # Agregar valores a la lista correspondiente en el diccionario
        if llave in medicos:
            medicos[llave].append([Disponibiidad, UltOcupado, lat, lon, NumSol, Inicioj, Finj])
        else:
            medicos[llave] = [int(Disponibiidad), int(UltOcupado),lat, lon, NumSol, int(Inicioj), int(Finj)]
    doctores = medicos
    
    return doctores, pacientes, Clientes

doctores = leer_datos()[0]
pacientes = leer_datos()[1]
Clientes = leer_datos()[2]

### Calcular la distancia/tiempo entre cada par de nodos

In [2]:
from math import radians, cos, sin, asin, sqrt
def distancia_tierra(lat1, lat2, lon1, lon2):
    
    # Convertir grados a radianes.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Formula Haversine, para calcular la distancia entre dos puntos de una esfera dadas sus coordenadas de longitud y latitud
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    
    # Radio de la tierra en km.
    r = 6371
    
    respuesta = ((c * r)/24.9)*60
    # Calcular la distancia final en Km.
    return respuesta

### Cambiar datos para Gantt

In [3]:
import datetime

def convertMin(minutos):
    # Obtenemos la fecha de hoy
    fecha_actual = datetime.date.today()
    # Combinamos la fecha de hoy con la hora 0:00:00
    fecha_hora = datetime.datetime.combine(fecha_actual, datetime.time.min) + datetime.timedelta(minutes=minutos)
    # Convertimos la fecha y hora a formato de cadena
    fecha_hora_str = fecha_hora.strftime("%Y-%m-%d %H:%M:%S")
    return fecha_hora_str

### Regla de Prioridad

In [4]:
# Regla de prioridad
def prioridad(NumSolicitudes, Tdesplazamiento, Mdescoupa):
    # Si el número de solicitudes esta entre 0 y 2
    if NumSolicitudes >= 0 and NumSolicitudes <= 2:
        beta = 0.55
        alpha = 0.45
    # Si el número de solicitudes esta entre 3 y 4
    elif NumSolicitudes >= 3 and NumSolicitudes <= 4:
        beta = 0.65
        alpha = 0.35
    # Si el número de solicitudes esta entre 5 o más
    elif NumSolicitudes >= 5:
        beta = 0.75
        alpha = 0.25

    # Calculamos la prioridad
    pesoPrioridad = NumSolicitudes * alpha + Tdesplazamiento + Mdescoupa * beta

    return pesoPrioridad

### Asignación de los médicos

In [5]:
global doctores

import pandas as pd

# Función para asignar doctor a paciente
def asignar_doctor(atencion,latitud, longitud, minuto, gantt, IDservicio, TipoCliente):

    # Lista de posible asignacion
    posibleAsignacion = []

    # Actualizar estado de los médicos
    if minuto > 0:
        for i in doctores:
            # Si el minuto actual es mayor al ultimo ocupado significa que el médico esta disponible
            if minuto > doctores[i][1]:
                # Actualizar estado del médico
                doctores[i][0] = 0
    
    for i in doctores:
        # Si el médico esta en su jornada laboral y esta disponible
        if minuto >= doctores[i][5] and minuto <= doctores[i][6] and doctores[i][0] == 0:
            # Agregar médico y su distancia a la lista de posible asignacion

            # Encontrar distancia de los médicos al paciente
            distancia = distancia_tierra(doctores[i][2], latitud, doctores[i][3], longitud)
            # Agregar médico y su distancia a la lista de posible asignacion
            posibleAsignacion.append([i, distancia])
    
    # Si no hay ningún médico disponible
    if len(posibleAsignacion) == 0:
        # Se buscan médicos que esten en su jornada laboral para asignarles el paciente usando la regla de prioridad

        for i in doctores:
            # Si el doctor esta en su jornada laboral
            if minuto >= doctores[i][5] and minuto <= doctores[i][6]:
                # Se aplican las reglas de prioridad

                # Calcular tiempo de desplazamiento
                Tdesplazamiento = distancia_tierra(doctores[i][2], latitud, doctores[i][3], longitud)
                # Calcular tiempo de desocupacion
                Mdesocupa = doctores[i][1]
                # Calcular prioridad
                pesoPrioridad = prioridad(doctores[i][4], Tdesplazamiento, Mdesocupa)
                # Agregar a la lista de posible asignacion
                posibleAsignacion.append([i, pesoPrioridad])
        

        # Si existen médicos en su jornada laboral
        if len(posibleAsignacion) > 0:
            # Escoger el médico con mayor prioridad, es decir, el que tenga menor peso

            # Filtrar a los médicos que cumplan con el tiempo de servicio
            for i in posibleAsignacion:
                # calcular el desplazamiento del medico
                Tdesplazamiento = distancia_tierra(doctores[i[0]][2], latitud, doctores[i[0]][3], longitud)

                # Si el médico no cumple con el tiempo de servicio se elimina de la lista de posible asignacion
                if minuto + Tdesplazamiento > minuto + Clientes[TipoCliente]:
                    #crear lista de médicos que no cumplen con el tiempo de servicio
                    listaMedicosNoCumplen = []
                    #agregar médicos que no cumplen con el tiempo de servicio
                    listaMedicosNoCumplen.append(i[0])
                    #eliminar médicos que no cumplen con el tiempo de servicio
                    posibleAsignacion.remove(i)

            # si existen médicos que cumplan con el tiempo de servicio
            if len(posibleAsignacion) > 0:
                # Ordenar la lista de posible asignacion
                posibleAsignacion.sort(key=lambda x: x[1])
                # Asignar el doctor con menor prioridad
                doc_Asignar = posibleAsignacion[0]

                # Actualizar variables del médico asignado
                Mdesocupa = doctores[doc_Asignar[0]][1]
                Tdesplazamiento = distancia_tierra(doctores[doc_Asignar[0]][2], latitud, doctores[doc_Asignar[0]][3], longitud)

                # Actualizar el gantt
                gantt.append({'Solicitud': IDservicio, 'Médico': doc_Asignar[0], 'Inicio': convertMin(Mdesocupa + Tdesplazamiento), 'Fin': convertMin(Mdesocupa + Tdesplazamiento + atencion), 'Distancia': Tdesplazamiento, 'Inicio Jornada': doctores[doc_Asignar[0]][5], 'Fin Jornada': doctores[doc_Asignar[0]][6], 'Atencion': atencion, 'Tipo de asignacion': 'Tipo 1'})

                # Actualizar estado del médico
                doctores[doc_Asignar[0]][0] = 1
                doctores[doc_Asignar[0]][1] = Mdesocupa + Tdesplazamiento + atencion
                doctores[doc_Asignar[0]][2] = latitud
                doctores[doc_Asignar[0]][3] = longitud
                doctores[doc_Asignar[0]][4] = doctores[doc_Asignar[0]][4] + 1

            # Si no existen médicos que cumplan con el tiempo de servicio
            else:
                # Calcular el médico que atienda la solicitud más rapido
                for i in listaMedicosNoCumplen:
                    # Calcular tiempo de desplazamiento
                    Tdesplazamiento = distancia_tierra(doctores[i][2], latitud, doctores[i][3], longitud)
                    # Calcular tiempo de desocupacion
                    Mdesocupa = doctores[i][1]
                    # Cambiar el peso de prioridad por el tiempo de desplazamiento
                    listaMedicosNoCumplen[i][1] = Mdesocupa + Tdesplazamiento

                # Ordenar la lista de posible asignacion
                listaMedicosNoCumplen.sort(key=lambda x: x[1])

                # Asignar el médico con menor tiempo de desplazamiento
                doc_Asignar = listaMedicosNoCumplen[0]

                # Actualizar variables del médico asignado
                Mdesocupa = doctores[doc_Asignar[0]][1]
                Tdesplazamiento = distancia_tierra(doctores[doc_Asignar[0]][2], latitud, doctores[doc_Asignar[0]][3], longitud)

                # Actualizar el gantt
                gantt.append({'Solicitud': IDservicio, 'Médico': doc_Asignar[0], 'Inicio': convertMin(Mdesocupa + Tdesplazamiento), 'Fin': convertMin(Mdesocupa + Tdesplazamiento + atencion), 'Distancia': Tdesplazamiento, 'Inicio Jornada': doctores[doc_Asignar[0]][5], 'Fin Jornada': doctores[doc_Asignar[0]][6], 'Atencion': atencion, 'Tipo de asignacion': 'Tipo 2'})

                # Actualizar estado del médico
                doctores[doc_Asignar[0]][0] = 1
                doctores[doc_Asignar[0]][1] = Mdesocupa + Tdesplazamiento + atencion
                doctores[doc_Asignar[0]][2] = latitud
                doctores[doc_Asignar[0]][3] = longitud
                doctores[doc_Asignar[0]][4] = doctores[doc_Asignar[0]][4] + 1
        else:
            # No hay médicos disponibles
            print("No hay médicos disponibles")
                
    # Si hay uno o más médicos disponibles
    elif len(posibleAsignacion) > 0:
        # Se filtrarán los médicos que no cumplan con el tiempo de servicio
        for i in posibleAsignacion:
            # Si el médico no cumple con el tiempo de servicio se elimina de la lista de posible asignacion
            if minuto + i[1] > minuto + Clientes[TipoCliente]:
                #crear lista de médicos que no cumplen con el tiempo de servicio
                listaMedicosNoCumplen = []
                #agregar médicos que no cumplen con el tiempo de servicio
                listaMedicosNoCumplen.append(i[0])
                #eliminar médicos que no cumplen con el tiempo de servicio
                posibleAsignacion.remove(i)
        
        # Si existen médicos que cumplan con el tiempo de servicio
        if len(posibleAsignacion) > 0:

            # calcular el peso de prioridad de cada médico
            for i in posibleAsignacion:
                # Calcular tiempo de desplazamiento
                Tdesplazamiento = distancia_tierra(doctores[i[0]][2], latitud, doctores[i[0]][3], longitud)
                # Calcular tiempo de desocupacion
                Mdesocupa = doctores[i[0]][1]
                # Calcular prioridad
                pesoPrioridad = prioridad(doctores[i[0]][4], Tdesplazamiento, Mdesocupa)

                # Cambiar el tiempo de desplazamiento por el peso de prioridad
                i[1] = pesoPrioridad
            
            # Ordenar la lista de posible asignacion
            posibleAsignacion.sort(key=lambda x: x[1])

            # Asignar el médico con menor prioridad
            doc_Asignar = posibleAsignacion[0]
            # Actualizar variables del médico asignado
            Mdesocupa = doctores[doc_Asignar[0]][1]
            Tdesplazamiento = distancia_tierra(doctores[doc_Asignar[0]][2], latitud, doctores[doc_Asignar[0]][3], longitud)

            # Actualizar el gantt
            gantt.append({'Solicitud': IDservicio, 'Médico': doc_Asignar[0], 'Inicio': convertMin(minuto + Tdesplazamiento), 'Fin': convertMin(minuto + Tdesplazamiento + atencion), 'Distancia': Tdesplazamiento, 'Inicio Jornada': doctores[doc_Asignar[0]][5], 'Fin Jornada': doctores[doc_Asignar[0]][6], 'Atencion': atencion, 'Tipo de asignacion': 'Tipo 3'})

            # Actualizar estado del médico
            doctores[doc_Asignar[0]][0] = 1
            doctores[doc_Asignar[0]][1] = minuto + Tdesplazamiento + atencion
            doctores[doc_Asignar[0]][2] = latitud
            doctores[doc_Asignar[0]][3] = longitud
            doctores[doc_Asignar[0]][4] = doctores[doc_Asignar[0]][4] + 1
        
        # Si no existen médicos que cumplan con el tiempo de servicio
        else:
            # Se asignará el médico con menor tiempo de desplazamiento
            for i in listaMedicosNoCumplen:
                # Calcular tiempo de desplazamiento
                Tdesplazamiento = distancia_tierra(doctores[i][2], latitud, doctores[i][3], longitud)
                # Cambiar el peso de prioridad por el tiempo de desplazamiento
                listaMedicosNoCumplen[i][1] = minuto + Tdesplazamiento
            
            # Ordenar la lista de posible asignacion
            listaMedicosNoCumplen.sort(key=lambda x: x[1])

            # Asignar el médico con menor tiempo de desplazamiento
            doc_Asignar = listaMedicosNoCumplen[0]

            # Actualizar variables del médico asignado
            Mdesocupa = doctores[doc_Asignar[0]][1]
            Tdesplazamiento = distancia_tierra(doctores[doc_Asignar[0]][2], latitud, doctores[doc_Asignar[0]][3], longitud)

            # Actualizar el gantt
            gantt.append({'Solicitud': IDservicio, 'Médico': doc_Asignar[0], 'Inicio': convertMin(minuto + Tdesplazamiento), 'Fin': convertMin(minuto + Tdesplazamiento + atencion), 'Distancia': Tdesplazamiento, 'Inicio Jornada': doctores[doc_Asignar[0]][5], 'Fin Jornada': doctores[doc_Asignar[0]][6], 'Atencion': atencion, 'Tipo de asignacion': 'Tipo 4'})

            # Actualizar estado del médico
            doctores[doc_Asignar[0]][0] = 1
            doctores[doc_Asignar[0]][1] = minuto + Tdesplazamiento + atencion
            doctores[doc_Asignar[0]][2] = latitud
            doctores[doc_Asignar[0]][3] = longitud
            doctores[doc_Asignar[0]][4] = doctores[doc_Asignar[0]][4] + 1

    return gantt

### Simular Pacientes

In [6]:
import pandas as pd
import openpyxl

global doctores

# Función principal para asignar médicos a pacientes
def asignacion_doctor1():
    pacientes = leer_datos()[1]
    gantt = []

    # Ordenar dict de pacientes por minuto
    pacientes = dict(sorted(pacientes.items(), key=lambda item: item[1][0]))

    for i in pacientes:
        # Asignar doctor a paciente
        atencion = pacientes[i][1]
        minuto = pacientes[i][0]
        latitud = pacientes[i][2]
        longitud = pacientes[i][3]
        IDservicio = i
        TipoCliente = pacientes[i][4]
        gantt = asignar_doctor(atencion,latitud,longitud, minuto, gantt, IDservicio, TipoCliente)
    
    return gantt

In [7]:
gantt = asignacion_doctor1()

### Gantt general

In [8]:
import plotly.express as px
import pandas as pd
#Poner los datos de la lista en un dataframe
df1 = pd.DataFrame(gantt)
# poner los datos en el formato que requiere plotly poner la columna solicitud como texto
df1['Solicitud'] = df1['Solicitud'].astype(str)
# poner la columna médico como texto
df1['Médico'] = df1['Médico'].astype(str)

fig = px.timeline(df1, x_start="Inicio", x_end="Fin", y="Médico", color="Médico",
                  hover_name="Solicitud",
                  title="Solicitudes atendidas por médicos")

fig.update_layout(xaxis_title="Tiempo", yaxis_title="Médico")
fig.update_yaxes(autorange="reversed")
fig.update_xaxes(rangeslider_visible=True)
# Hacer el grafico más grande para que se vea mejor
fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
)
fig.show()

In [11]:
import folium
from folium import plugins
from folium.plugins import HeatMap

# Crear un mapa centrado en la ciudad de Bogotá
mapa = folium.Map(location=[4.60971, -74.08175], zoom_start=12)

# Crear dict de coordenadas pacientes
coordenadas = {}

for i in pacientes:
    coordenadas[i] = [pacientes[i][2], pacientes[i][3]]

# Crear un mapa de calor con la ubicación de los pacientes
HeatMap(data=coordenadas.values(), radius=15).add_to(mapa)

# Crear un grupo de marcadores de pacientes
markers = folium.FeatureGroup(name='Pacientes')

# Agregar el marcador de cada paciente al grupo de marcadores
for i in pacientes:
    folium.Marker(
        location=[pacientes[i][2], pacientes[i][3]],
        popup="Paciente " + str(i),
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(markers)

# Crear un control de capas para mostrar u ocultar los marcadores de pacientes
folium.LayerControl().add_to(mapa)

# Mostrar el mapa
mapa

In [10]:
# Encontrar la cantidad de solicitudes atendidas por cada médico
def solicitudes_atendidas(gantt):
    solicitudes = {}

    for i in gantt:
        if i['Médico'] not in solicitudes:
            solicitudes[i['Médico']] = 1
        else:
            solicitudes[i['Médico']] += 1

    return solicitudes

solicitudes = solicitudes_atendidas(gantt)
print(solicitudes)

# Calcular la desviación estándar de las solicitudes atendidas por cada médico
def desviacion_estandar(solicitudes):
    # Calcular la media de las solicitudes atendidas por cada médico
    media = sum(solicitudes.values())/len(solicitudes)

    # Calcular la varianza de las solicitudes atendidas por cada médico
    varianza = sum([(i - media)**2 for i in solicitudes.values()])/len(solicitudes)

    # Calcular la desviación estándar de las solicitudes atendidas por cada médico
    desviacion = varianza**(1/2)

    return desviacion

desviacion = desviacion_estandar(solicitudes)
print(desviacion)

# calcular solicited promedio por médico
def solicitudes_promedio(solicitudes):
    promedio = sum(solicitudes.values())/len(solicitudes)
    return promedio

promedio = solicitudes_promedio(solicitudes)
print(promedio)

# Calcular total de solicitudes atendidas
def total_solicitudes(solicitudes):
    total = sum(solicitudes.values())
    return total

total = total_solicitudes(solicitudes)
print(total)

{1032491705: 5, 1015405667: 7, 1018472151: 10, 1015437933: 10, 80073352: 9, 52997773: 11, 79955886: 11, 1015414697: 7, 1019088914: 8, 1020777651: 8, 1018446151: 9, 80185764: 7, 39779707: 8, 52200795: 1, 1127250183: 9, 1024468225: 9, 1121853934: 7, 1016039086: 7, 80383487: 7, 1020803066: 7, 57293715: 8, 1020808271: 9, 80727764: 8, 1098635342: 8, 1082996581: 7, 1117504115: 9, 1085310672: 8, 1095825225: 8, 1014217039: 8, 1014266018: 8, 1098697055: 10, 85488148: 8, 80075437: 9, 1053327980: 7, 1083026203: 7, 1083012532: 6, 1018440480: 7, 1019074166: 7, 1140888504: 7}
1.6876392484004448
7.846153846153846
306
